### 1 Creating the notebook and importing library BeautifulSoup for webscrapping

In [131]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

### 2 Scrapping the webpage


In [132]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
page

<Response [200]>

In [133]:
soup = BeautifulSoup(page.content,'html.parser')

### 2.1Finding the table location and then accessing each column of each row

In [134]:
table = soup.find('table',{'class':'wikitable sortable'})
table_rows = table.find_all('tr')

### 2.2 Creating a pandas DataFrame with the required 3 columns

In [135]:
data = []
for row in table_rows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])

### 3 Processing the cells that have an assigned borough and Postal Code is Null

In [136]:
df = df[~df['PostalCode'].isnull()] 
df = df[df['Borough'] != 'Not assigned']

### 3.1 Combining Neighbourhoods

In [137]:
df["Neighbourhood"] = df.groupby("PostalCode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))
#remove duplicates
df = df.drop_duplicates()
#update index to be postcode if it isn't already
if(df.index.name != 'PostalCode'):
    df = df.set_index('PostalCode')   


### 3.2 Assigning the neighborhood as the borough in case it is not assigned

In [138]:
df = df.reset_index()
df['Neighbourhood'].replace("Not assigned", df["Borough"],inplace=True)
df.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [139]:
!wget -q -O "toronto_coordinates.csv" "http://cocl.us/Geospatial_data"
coords = pd.read_csv('toronto_coordinates.csv')

In [140]:
coords.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
toronto_df_temp = df.set_index('PostalCode')
coords_temp = coords.set_index('Postal Code')
toronto_df_coords = pd.concat([toronto_df_temp, coords_temp], axis=1, join='inner')
toronto_df_coords.index.name = 'PostalCode'
toronto_df_coords.reset_index(inplace=True)
print(toronto_df_coords.shape)
toronto_df_coords.head()

(99, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Question No 3 Exploring and clustering the neighborhoods in Toronto

In [23]:
from geopy.geocoders import Nominatim 
import requests # library to handle requests
from pandas.io.json import json_normalize 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.4.5.1         |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ca-certificates-2020.4.5.1 |       hecc5488_0         146 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                       

In [141]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df_coords['Borough'].unique()),
        toronto_df_coords.shape[0]
    )
)

The dataframe has 10 boroughs and 99 neighborhoods.


In [142]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="tl-toronto-neigh")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 3.1 Creating a map of Toronto with neighborhoods superimposed on top

In [162]:
##create map of Toronoto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df_coords['Latitude'], toronto_df_coords['Longitude'], toronto_df_coords['Borough'], toronto_df_coords['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### 3.2 Selecting the Boroughs with the word Toronto(4 of them)

In [163]:
toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
toronto_central_df = toronto_df_coords[toronto_df_coords['Borough'].isin(toronto_boroughs)].reset_index(drop=True)
#toronto_central_df.drop(['PostalCode'],axis=1, inplace=True)
print(toronto_central_df.shape)
toronto_central_df.head()

(39, 5)


,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [164]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)
for lat, long, post, borough, neigh in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['PostalCode'], toronto_central_df['Borough'], toronto_central_df['Neighbourhood']):
    label = '{}, {},{}'.format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
                    [lat, long],
                    radius=5,
                    popup=popup,
                    color='red',
                    fill=True,
                    fill_color='yellow',
                    parse_html=False).add_to(map_toronto)
       
map_toronto

### 3.3 Defining Foursquare Credentials and Version

In [165]:
CLIENT_ID = 'J4HSDA4XCLUXAZRLJCNUKLTHA4KGL0YCDJ4XOENLLDZG5EQW' # your Foursquare ID
CLIENT_SECRET = 'D2I4CVBH54SM0E5BPYLIRLDGEFEJTKDODBW45UFZLB0HQYRT' # your Foursquare Secret
VERSION = '20200525' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J4HSDA4XCLUXAZRLJCNUKLTHA4KGL0YCDJ4XOENLLDZG5EQW
CLIENT_SECRET:D2I4CVBH54SM0E5BPYLIRLDGEFEJTKDODBW45UFZLB0HQYRT


### 3.4 Obtaining the top 100 venues that are within a radius of 500 meters.

In [166]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(toronto_central_df['Latitude'], toronto_central_df['Longitude'], toronto_central_df['PostalCode'], toronto_central_df['Borough'], toronto_central_df['Neighbourhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [167]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighbourhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1616, 9)


,PostalCode,Borough,Neighbourhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa


In [168]:
venues_df.groupby(['PostalCode', 'Borough', 'Neighbourhood'])['VenueName'].count()

PostalCode  Borough           Neighbourhood                                                                                             
M4E         East Toronto      The Beaches                                                                                                     4
M4K         East Toronto      The Danforth West, Riverdale                                                                                   42
M4L         East Toronto      India Bazaar, The Beaches West                                                                                 23
M4M         East Toronto      Studio District                                                                                                41
M4N         Central Toronto   Lawrence Park                                                                                                   3
M4P         Central Toronto   Davisville North                                                                                                9

In [169]:
len(venues_df['VenueCategory'].unique())

237

### 3.5 Analyzing each neighbourhood

In [170]:
toronto_central_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")
# add postal, borough and neighborhood column back to dataframe
toronto_central_onehot['PostalCode'] = venues_df['PostalCode']
toronto_central_onehot['Borough'] = venues_df['Borough'] 
toronto_central_onehot['Neighborhoods'] = venues_df['Neighbourhood']
# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_central_onehot.columns[-3:]) + list(toronto_central_onehot.columns[:-3])
toronto_central_onehot = toronto_central_onehot[fixed_columns]
print(toronto_central_onehot.shape)
toronto_central_onehot.head()

(1616, 240)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.6 Next, Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [171]:
toronto_central_venues_freq = toronto_central_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(toronto_central_venues_freq.shape)
toronto_central_venues_freq

(39, 240)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.250000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.023810,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.023810
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.024390,0.000000,0.00000,0.024390
4,M4N,Central Toronto,Lawrence Park,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.050000
7,M4S,Central Toronto,Davisville,0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.027778,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.333333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.0000,0.0000,0.0000,0.000,0.000,0.000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.00000,0.000000


### 3.7 Fetching 10 most occurance venue types in each area

In [154]:
import numpy as np
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

### 3.8 Create a new dataframe
    

In [156]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_central_venues_freq['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_central_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_central_venues_freq['Neighborhoods']

for ind in np.arange(toronto_central_venues_freq.shape[0]):
    row_categories = toronto_central_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Airport Gate
31,M6H,West Toronto,"Dufferin, Dovercourt Village",Bakery,Pharmacy,Music Venue,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Grocery Store
32,M6J,West Toronto,"Little Portugal, Trinity",Bar,Restaurant,Asian Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Men's Store,Cuban Restaurant,Brewery,Record Shop
26,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",Café,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Gaming Cafe,Dessert Shop,Bar,Vegetarian / Vegan Restaurant,Caribbean Restaurant
33,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
15,M5C,Downtown Toronto,St. James Town,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Italian Restaurant,Restaurant,Beer Bar,Moroccan Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
25,M5S,Downtown Toronto,"University of Toronto, Harbord",Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Chinese Restaurant,Dessert Shop,Pub
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",Café,Sandwich Place,Coffee Shop,BBQ Joint,History Museum,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant,Cosmetics Shop
14,M5B,Downtown Toronto,"Garden District, Ryerson",Clothing Store,Coffee Shop,Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant


### 3.9 Using K-means Clustering to cluster the area

In [160]:
from sklearn.cluster import KMeans
kclusters = 4
toronto_central_venues_freq_clustering = toronto_central_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)
kmeans = KMeans(n_clusters=kclusters).fit(toronto_central_venues_freq_clustering)
toronto_central_clustered_df = toronto_central_df
toronto_central_clustered_df['Cluster'] = kmeans.labels_
toronto_central_clustered_df = toronto_central_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
toronto_central_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
toronto_central_clustered_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.628947,-79.394420,0,Airport Lounge,Airport Service,Airport Terminal,Sculpture Garden,Boat or Ferry,Rental Car Location,Plane,Coffee Shop,Harbor / Marina,Airport Gate
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259,0,Bakery,Pharmacy,Music Venue,Middle Eastern Restaurant,Bank,Bar,Café,Pool,Supermarket,Grocery Store
11,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750,0,Bar,Restaurant,Asian Restaurant,Coffee Shop,Vegetarian / Vegan Restaurant,Café,Men's Store,Cuban Restaurant,Brewery,Record Shop
30,M5T,Downtown Toronto,"Kensington Market, Chinatown, Grange Park",43.653206,-79.400049,0,Café,Bakery,Coffee Shop,Vietnamese Restaurant,Mexican Restaurant,Gaming Cafe,Dessert Shop,Bar,Vegetarian / Vegan Restaurant,Caribbean Restaurant
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191,0,Café,Coffee Shop,Breakfast Spot,Grocery Store,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store,Bakery
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Café,Coffee Shop,Cocktail Bar,Gastropub,American Restaurant,Clothing Store,Italian Restaurant,Restaurant,Beer Bar,Moroccan Restaurant
17,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Gastropub,Brewery,Bakery,American Restaurant,Yoga Studio,Neighborhood,Sandwich Place,Cheese Shop
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,0,Café,Restaurant,Bar,Italian Restaurant,Japanese Restaurant,Bookstore,Bakery,Chinese Restaurant,Dessert Shop,Pub
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,0,Café,Sandwich Place,Coffee Shop,BBQ Joint,History Museum,Indian Restaurant,Donut Shop,Liquor Store,Middle Eastern Restaurant,Cosmetics Shop
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Restaurant,Bubble Tea Shop,Café,Japanese Restaurant,Italian Restaurant,Cosmetics Shop,Middle Eastern Restaurant,Fast Food Restaurant


### 3.10 Displaying Using Map

In [172]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_central_clustered_df['Latitude'], toronto_central_clustered_df['Longitude'], toronto_central_clustered_df['PostalCode'], toronto_central_clustered_df['Borough'], toronto_central_clustered_df['Neighbourhood'], toronto_central_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters